In [1]:
pip install mysql-connector-python numpy pandas 

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install pymysql sqlalchemy

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 445.3 kB/s eta 0:00:00


In [9]:
import os
from sqlalchemy import create_engine, text
import pandas as pd
from sqlalchemy.types import Integer, Float, Boolean, DateTime, Text

In [10]:
USER     = 'root'
PASSWORD = 'navya'
HOST     = 'localhost'
DB       = 'ecommerce'

In [11]:
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DB}"
    "?charset=utf8mb4",        
    pool_pre_ping=True         
)

In [12]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.scalar())

1


In [13]:
def sql_dtype_mapping(series: pd.Series):
    if pd.api.types.is_integer_dtype(series):
        return Integer()
    elif pd.api.types.is_float_dtype(series):
        return Float()
    elif pd.api.types.is_bool_dtype(series):
        return Boolean()
    elif pd.api.types.is_datetime64_any_dtype(series):
        return DateTime()
    else:
        return Text()


In [14]:
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items') 
]

In [17]:
folder_path = './dataset/'

In [18]:
for csv_name, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_name)
    print(f"Loading {csv_name} → {table_name}…")

    df = pd.read_csv(file_path, parse_dates=True)
    df.columns = [c.strip().replace(' ', '_').replace('-', '_').replace('.', '_')
                  for c in df.columns]

    # build a dtype dict for to_sql
    dtype_dict = {col: sql_dtype_mapping(df[col]) for col in df.columns}

    # this will CREATE TABLE if needed, then INSERT in chunks
    df.to_sql(
        name=table_name,
        con=engine,
        if_exists='replace',   # or 'append'
        index=False,
        dtype=dtype_dict,
        chunksize=1000         # adjust up/down based on memory
    )
    print(f" → done ({len(df)} rows)\n")

Loading customers.csv → customers…
 → done (99441 rows)

Loading orders.csv → orders…
 → done (99441 rows)

Loading sellers.csv → sellers…
 → done (3095 rows)

Loading products.csv → products…
 → done (32951 rows)

Loading geolocation.csv → geolocation…
 → done (1000163 rows)

Loading payments.csv → payments…
 → done (103886 rows)

Loading order_items.csv → order_items…
 → done (112650 rows)

